<a href="https://colab.research.google.com/github/26248-cmd/03-Torsin-coding68/blob/main/03_%E0%B8%A3%E0%B8%B0%E0%B8%9A%E0%B8%9A%E0%B8%88%E0%B8%B1%E0%B8%94%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%9E%E0%B8%A5%E0%B8%B1%E0%B8%87%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%AD%E0%B8%B1%E0%B8%88%E0%B8%89%E0%B8%A3%E0%B8%B4%E0%B8%A2%E0%B8%B0_(Smart_Energy_Manager)_%E0%B9%80%E0%B8%A7%E0%B8%AD%E0%B8%A3%E0%B9%8C%E0%B8%8A%E0%B8%B1%E0%B8%99%E0%B8%A3%E0%B8%B0%E0%B8%9A%E0%B8%9A%E0%B9%80%E0%B8%A3%E0%B8%B4%E0%B9%88%E0%B8%A1%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%97%E0%B8%B3%E0%B8%87%E0%B8%B2%E0%B8%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ⚡ ระบบจัดการพลังงานอัจฉริยะ (Smart Energy Manager) - เวอร์ชันระบบเริ่มการทำงาน
# @markdown กดรันเซลล์นี้เพื่อเริ่มต้นใช้งานระบบ (อ้างอิงเวลาเขตประเทศไทย Asia/Bangkok)

import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import random
import pandas as pd
import time
import threading

try:
    import pytz
except ImportError:
    !pip install -q pytz
    import pytz

# --- Basic Data Configuration ---
MIN_UNIT_PRICE = 3.99
history_data = []
TH_TZ = pytz.timezone('Asia/Bangkok')
is_monitoring = False # Monitor state

SAVING_TIPS = [
    "ปิดไฟดวงที่ไม่ใช้งานทุกครั้ง",
    "ตั้งอุณหภูมิแอร์ที่ 25-26 องศาเซลเซียส",
    "ถอดปลั๊กเครื่องใช้ไฟฟ้าหลังเลิกใช้งาน",
    "ล้างแอร์สม่ำเสมอทุก 6 เดือน",
    "เปิดม่านรับแสงธรรมชาติแทนการเปิดไฟในช่วงกลางวัน"
]

# --- Main Functions ---

def get_now_th():
    """Get current time in Thailand timezone"""
    return datetime.now(TH_TZ)

def check_and_save(is_auto=False):
    """Function to calculate and save power data"""
    try:
        limit = float(max_power.value)
        used = float(used_power.value)
        target_time = f"{hour_select.value:02d}:{min_select.value:02d}"
        now = get_now_th()

        with output_area:
            if is_auto:
                print(f"\n🔔 [Auto-Save Active] at {now.strftime('%H:%M:%S')}")
            else:
                clear_output()
                print(f"--- Analysis Report: {now.strftime('%d/%m/%Y %H:%M:%S')} (TH Time) ---")

            # Progress Bar logic
            percent = (used / limit) * 100
            bar_len = 20
            fill = int(min(bar_len, bar_len * used // limit))
            bar = '█' * fill + '-' * (bar_len - fill)
            color_emoji = "🟢" if percent <= 80 else "🟡" if percent <= 100 else "🔴"
            print(f"Power Usage Level: [{bar}] {percent:.1f}% {color_emoji}")

            # Money calculation (Negative if over limit)
            diff_units = limit - used
            money_status = diff_units * MIN_UNIT_PRICE

            if used > limit:
                print(f"⚠️ Usage exceeded by {abs(diff_units):.2f} kWh | Cost: -{abs(money_status):.2f} THB")
            else:
                print(f"✅ Savings: {diff_units:.2f} kWh | Saved: {money_status:.2f} THB")

            # Append to history
            history_data.append({
                'timestamp': now,
                'year': now.year,
                'month': now.strftime("%B %Y"),
                'limit': limit,
                'used': used,
                'money': money_status,
                'alert_at': target_time
            })
            print("✅ Data successfully saved to history.")

            if not is_auto and used <= limit:
                print(f"✨ Tip: {random.choice(SAVING_TIPS)}")

    except Exception as e:
        with output_area:
            print(f"❌ Error: {e}")

def monitor_loop():
    """Background loop for checking time"""
    global is_monitoring
    last_checked_minute = -1

    while is_monitoring:
        now = get_now_th()
        # Trigger only once per minute
        if now.minute != last_checked_minute:
            if now.hour == hour_select.value and now.minute == min_select.value:
                check_and_save(is_auto=True)
                last_checked_minute = now.minute
        time.sleep(10)

def toggle_monitor(b):
    """Start or Stop the monitoring system"""
    global is_monitoring
    if not is_monitoring:
        is_monitoring = True
        status_label.value = "<b style='color: #27ae60;'>● ระบบกำลังทำงาน (Active)</b>"
        btn_monitor.button_style = 'danger'
        btn_monitor.description = "หยุดการทำงาน"
        btn_monitor.icon = 'stop'

        # Start background thread
        threading.Thread(target=monitor_loop, daemon=True).start()

        with output_area:
            print(f"🚀 ระบบเริ่มเฝ้าระวัง... จะทำการบันทึกอัตโนมัติเมื่อถึงเวลา {hour_select.value:02d}:{min_select.value:02d}")
    else:
        is_monitoring = False
        status_label.value = "<b style='color: #c0392b;'>○ ระบบหยุดทำงาน (Inactive)</b>"
        btn_monitor.button_style = 'success'
        btn_monitor.description = "เริ่มระบบเฝ้าระวัง"
        btn_monitor.icon = 'play'
        with output_area:
            print("🛑 ระบบเฝ้าระวังหยุดทำงานแล้ว")

def on_calc_click(b):
    check_and_save(is_auto=False)

def show_history(b):
    with output_area:
        clear_output()
        if not history_data:
            print("📂 No history records found.")
            return

        df = pd.DataFrame(history_data).sort_values(by=['timestamp'], ascending=False)
        print("📊 --- Transaction History (Thailand Time) ---")

        # Monthly Summary Calculation logic
        monthly_groups = df.groupby('month')

        for _, row in df.iterrows():
            status_tag = "✅ Saved" if row['money'] >= 0 else "❌ Over"
            money_val = f"{row['money']:.2f}"
            print(f"    [{row['timestamp'].strftime('%d/%m %H:%M')}] Used: {row['used']:.2f} kWh | {status_tag}: {money_val} THB")

        print("\n📈 --- Monthly Summary ---")
        for month, group in monthly_groups:
            count = len(group)
            total_used = group['used'].sum()
            total_limit = group['limit'].sum()
            total_savings = (total_limit - total_used) * MIN_UNIT_PRICE

            print(f"📅 Month: {month}")
            print(f"   - Average: {total_used/count:.2f} kWh/day")
            if total_savings >= 0:
                print(f"   - 💰 Total Saved: {total_savings:.2f} THB")
            else:
                print(f"   - 💸 Total Loss: {abs(total_savings):.2f} THB")

# --- UI Setup ---
style = {'description_width': 'initial'}

hour_select = widgets.Dropdown(options=[(f"{i:02d}", i) for i in range(24)], value=18, description='ชั่วโมง:', layout={'width': '80px'})
min_select = widgets.Dropdown(options=[(f"{i:02d}", i) for i in range(60)], value=0, description='นาที:', layout={'width': '80px'})
max_power = widgets.FloatText(description="ขีดจำกัดไฟ/วัน (kWh):", style=style, value=20.0)
used_power = widgets.FloatText(description="ไฟที่ใช้จริงวันนี้ (kWh):", style=style, value=0.0)

status_label = widgets.HTML(value="<b style='color: #c0392b;'>○ ระบบหยุดทำงาน (Inactive)</b>")

btn_calc = widgets.Button(description="บันทึกแมนนวล", button_style='primary', icon='edit', layout={'width': '150px'})
btn_monitor = widgets.Button(description="เริ่มระบบเฝ้าระวัง", button_style='success', icon='play', layout={'width': '180px'})
btn_history = widgets.Button(description="ดูประวัติและสรุปผล", button_style='info', icon='history', layout={'width': '180px'})

output_area = widgets.Output()

# Event Handlers
btn_calc.on_click(on_calc_click)
btn_history.on_click(show_history)
btn_monitor.on_click(toggle_monitor)

# Layout Construction
print("╔════════════════════════════════════════════════════════╗")
print("║       Energy Saver Smart Monitor (Version 2.0)         ║")
print("╚════════════════════════════════════════════════════════╝")

ui_header = widgets.HBox([status_label], layout={'justify_content': 'center', 'margin': '10px'})
input_grid = widgets.VBox([
    widgets.HBox([max_power, used_power]),
    widgets.HBox([widgets.Label("ตั้งเวลาบันทึกอัตโนมัติ:"), hour_select, widgets.Label(":"), min_select]),
    widgets.HBox([btn_monitor, btn_calc, btn_history])
], layout={'padding': '15px', 'border': '1px solid #ddd', 'border_radius': '10px'})

display(ui_header, input_grid, output_area)

╔════════════════════════════════════════════════════════╗
║       Energy Saver Smart Monitor (Version 2.0)         ║
╚════════════════════════════════════════════════════════╝


Output()